# Sequential Testing of Contexts
This notebook will provide a quick demonstration on how to use the sequential testing function and interpret the results.

In [9]:
import numpy as np
import pandas as pd
from contextualized.analysis.pvals import test_sequential_contexts
from contextualized.easy import ContextualizedRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

## Gather Data for Training

In [10]:
X, Y = load_diabetes(return_X_y=True, as_frame=True)
Y = np.expand_dims(Y.values, axis=-1)
C = X[['age', 'sex', 'bmi']]
X.drop(['age', 'sex', 'bmi'], axis=1, inplace=True)

seed = 1
C_train, C_test, X_train, X_test, Y_train, Y_test = train_test_split(C, X, Y, test_size=0.20, random_state=seed)

In [11]:
# converting to pandas dataframe
C_train_df = pd.DataFrame(C_train)
X_train_df = pd.DataFrame(X_train)
Y_train_df = pd.DataFrame(Y_train)

## Using the sequential testing function with a ContextualizedRegressor

In [12]:
%%capture
pvals = test_sequential_contexts(ContextualizedRegressor, C_train_df, X_train_df, Y_train_df, encoder_type="mlp", max_epochs=3, learning_rate=1e-2, n_bootstraps=10)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type           | Params
---------------------------------------------
0 | metamodel | NaiveMetamodel | 1.4 K 
---------------------------------------------
1.4 K     Trainable params
0         Non-trainable params
1.4 K     Total params
0.006     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=3` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type           | Params
---------------------------------------------
0 | metamodel | NaiveMetamodel | 1.4 K 
---------------------------------------------
1.4 K     Trainable params
0         Non-trainable params
1.4 K     Total params
0.006     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=3` reached.
GPU av

## Analyzing results

Below, the displayed p-values are found from the sequential testing of context features on predictor variables for a given target variable. These p-values are calculated based on the consistency of the sign of effects across multiple bootstraps. A p-value close to 0 suggests a strong consistency and significance in the effect, indicating whether the relationship is consistently positive or negative. This analysis enables us to quantify the uncertainty associated with the bootstrap confidence intervals.

In [13]:
pvals

,Context,Predictor,Target,Pvals
0,age,bp,0,0.090909
1,age,s1,0,0.090909
2,age,s2,0,0.090909
3,age,s3,0,0.090909
4,age,s4,0,0.090909
5,age,s5,0,0.090909
6,age,s6,0,0.090909
7,sex,bp,0,0.181818
8,sex,s1,0,0.181818
9,sex,s2,0,0.363636
